In [195]:
import numpy as np
import pandas as pd

## Content based recommendation system

In [196]:
df=pd.read_csv('result_final.csv')

In [197]:
df.head()

,Unnamed: 0.1,Unnamed: 0,link,text,title,date,keywords,summary,title_summary
0,0,0,http://techcrunch.com/2020/09/07/vodafone-idea...,"Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...,2020-09-07 00:00:00,"['rebrands', 'idea', 'vi', 'giant', 'brand', '...","Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...
1,1,1,http://techcrunch.com/2020/09/16/facebook-addr...,"At the beginning of the previous decade, Faceb...",Facebook addresses political controversy in In...,2020-09-16 00:00:00,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...
2,2,2,http://techcrunch.com/2020/09/14/youtube-launc...,"As TikTok’s fate in the U.S. remains murky, Yo...","YouTube launches its TikTok rival, YouTube Sho...",2020-09-14 00:00:00,"['rival', 'video', 'feature', 'access', 'youtu...","As TikTok’s fate in the U.S. remains murky, Yo...","YouTube launches its TikTok rival, YouTube Sho..."
3,3,3,http://techcrunch.com/2020/09/09/groww-an-inve...,Even as more than 150 million people are using...,"Groww, an investment app for millennials in In...",2020-09-09 00:00:00,"['world', 'yc', 'continuity', 'stocks', 'groww...","YC Continuity, the growth-stage investment fun...","Groww, an investment app for millennials in In..."
4,4,4,http://techcrunch.com/2020/09/15/lanzatech-is-...,As part of the continuing global rollout of La...,LanzaTech is developing a small-scale waste bi...,2020-09-15 00:00:00,"['production', 'distributed', 'developing', 's...",As part of the continuing global rollout of La...,LanzaTech is developing a small-scale waste bi...


In [198]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2190 entries, 0 to 2189
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0.1   2190 non-null   int64 
 1   Unnamed: 0     2190 non-null   int64 
 2   link           2190 non-null   object
 3   text           2184 non-null   object
 4   title          2187 non-null   object
 5   date           1691 non-null   object
 6   keywords       2187 non-null   object
 7   summary        2184 non-null   object
 8   title_summary  2184 non-null   object
dtypes: int64(2), object(7)
memory usage: 154.1+ KB


In [199]:
df.drop(columns=['Unnamed: 0.1','link','date'],axis=1,inplace=True)

In [200]:
def string(x):
    x=x[1:-1]
    s=''
    for i in x.split(','):
       s+=i+','
    return s

In [201]:
df.fillna(method='ffill',inplace=True)

In [202]:
df['keywords']=df['keywords'].agg(string)

In [203]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(stop_words='english')

tfidf_mat=tfidf.fit_transform(df['keywords'])
tfidf_mat

<2190x8023 sparse matrix of type '<class 'numpy.float64'>'
	with 32479 stored elements in Compressed Sparse Row format>

In [204]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim=linear_kernel(tfidf_mat,tfidf_mat)
cosine_sim

array([[1.        , 0.05450556, 0.03955338, ..., 0.        , 0.        ,
        0.        ],
       [0.05450556, 1.        , 0.07178309, ..., 0.        , 0.        ,
        0.        ],
       [0.03955338, 0.07178309, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [205]:
df1=pd.DataFrame(pd.Series(df['keywords'],index=df.index))
df1

,keywords
0,"'rebrands', 'idea', 'vi', 'giant', 'brand', 'l..."
1,"'opportunities', 'whatsapp', 'controversy', 'r..."
2,"'rival', 'video', 'feature', 'access', 'youtub..."
3,"'world', 'yc', 'continuity', 'stocks', 'groww'..."
4,"'production', 'distributed', 'developing', 'sc..."
...,...
2185,"'bolsonaro', 'guedes', 'reins', 'economic', 'f..."
2186,"'push', 'bank', 'england', 'zero', 'signal', '..."
2187,"'shortages', 'imports', 'gasoline', 'shipments..."
2188,"'international', 'tvs', 'sundays', 'united', '..."


In [206]:
import re
def recomendation(x):
    ind=df1[df1['keywords'].str.contains(x,flags=re.IGNORECASE,regex=True)].index[0]
    sim_score=list(enumerate(cosine_sim[ind]))
    sim_score=sorted(sim_score,key=lambda a:a[1],reverse=True)
    sim_score=sim_score[1:8]
    final_ind=[i[0] for i in sim_score]
    return final_ind 
    

In [207]:
r=input('Enter keyword for news title:')
ind=recomendation(r)

print('Recommended news are:')
for i in ind:
    print('>>',df['title'][i],'\n \t>>>',df['title_summary'][i])

Enter keyword for news title:father
Recommended news are:
>> Motor racing: No 'Andretti Curse' in August, says Marco 
 	>>> Motor racing: No 'Andretti Curse' in August, says Marco (Reuters) - After more than five decades of Brickyard frustration the “Andretti Curse” has become as much a part of the Indianapolis 500 as the command for drivers to “start your engines”.

Aug 17, 2020; Indianapolis, Indiana, USA; Indy Car Series driver Marco Andretti poses for a family photo shoot with wife Marta and his sister and his father Michael Andretti and Michael's family during a front row photo shoot before the 104th running of the Indianapolis 500 at the Indianapolis Motor Speedway. Mandatory Credit: Brian Spurlock-USA TODAY Sports

American motor racing royalty, the Andrettis have ruled over open wheel racing in the United States with a string of victories that have connected generations -- father (Mario), to son (Michael), to grandson (Marco).

But for all their success at circuits around the w

## Collaborative filtering recommendation system

In [208]:
df2=pd.read_csv('Netflix_Dataset_Movie.csv')
df3=pd.read_csv('Netflix_Dataset_Rating.csv')

In [209]:
df3.head()

,User_ID,Rating,Movie_ID
0,712664,5,3
1,1331154,4,3
2,2632461,3,3
3,44937,5,3
4,656399,4,3


In [210]:
users=df3['User_ID'].unique().shape[0]
movie=df3['Movie_ID'].unique().shape[0]

In [211]:
df3.sort_values('User_ID',inplace=True)

In [212]:
user_uni

array([      6,       7,      79, ..., 2649388, 2649426, 2649429],
      dtype=int64)

In [213]:
df3

,User_ID,Rating,Movie_ID
12905688,6,4,3423
14442278,6,2,3817
1480634,6,3,361
4883971,6,3,1314
4854218,6,1,1307
...,...,...,...
1748661,2649429,1,457
16737204,2649429,4,4356
9038631,2649429,3,2395
10401049,2649429,5,2782


In [215]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
df3['User_ID']=le.fit_transform(df3['User_ID'])

In [216]:
df3.sort_values('Movie_ID',inplace=True)
df3['Movie_id']=le.fit_transform(df3['Movie_ID'])

In [218]:
a=np.zeros((users,movie))
for i in df3.itertuples():
    a[i[1]-1,i[4]-1]=i[2]
a

array([[5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 4., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [219]:
from scipy.sparse import csr_matrix
sparse_matrix=csr_matrix(a)

In [220]:
from sklearn.neighbors import NearestNeighbors

nn=NearestNeighbors(n_neighbors=5,algorithm='brute',metric='cosine',n_jobs=-1)
nn.fit(sparse_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [221]:
df3

,User_ID,Rating,Movie_ID,Movie_id
1339,31392,3,3,0
760,132149,4,3,0
1444,124350,4,3,0
1269,140183,4,3,0
544,40812,4,3,0
...,...,...,...,...
17333648,51660,3,4496,1349
17336201,112223,4,4496,1349
17335333,114977,4,4496,1349
17331273,118028,5,4496,1349


In [223]:
data=df3.sort_values('User_ID',ascending=True)
colab_filter=data[data['User_ID']==69].Movie_ID
colab_filter=colab_filter.tolist()
len(colab_filter)

137

In [227]:
item_id=[]
for i in colab_filter:
    dist,ind=nn.kneighbors(sparse_matrix[i],n_neighbors=5)
    ind=ind.flatten()
    ind=ind[:1]
    item_id.extend(ind)
item_id=item_id[:10]

In [228]:
item_id

[197, 3161, 607, 3938, 4216, 2128, 3579, 2080, 3670, 3907]

In [229]:
df2

,Movie_ID,Year,Name
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW
...,...,...,...
17765,17766,2002,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004,Fidel Castro: American Experience
17767,17768,2000,Epoch
17768,17769,2003,The Company


In [230]:
print("Movies recommended by User 69:")
for i in item_id:
    print(">>>>",df2['Name'][i-1])

Movies recommended by User 69:
>>>> Taking Lives
>>>> Scary Movie 2
>>>> Speed
>>>> Shrek 2
>>>> Jurassic Park III
>>>> Rudy
>>>> 2 Fast 2 Furious
>>>> Holiday in the Sun
>>>> Encino Man
>>>> Madeline
